In [1]:
!pip install transformers==4.28.0 evaluate sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2


In [2]:
text = "translate English to French: Legumes share resources with nitrogen-fixing bacteria."

In [3]:
from transformers import pipeline

translator = pipeline("translation_en_to_fr", model="ethansimrm/t5_small_prelim_emea_enfr") #242MB in size
translator(text)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

[{'translation_text': 'Les légumes partagent les ressources avec les bactéries fixatrices de l’ azote.'}]

In [4]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value = user_secrets.get_secret("HF_KEY")
login(secret_value)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid.
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
from datasets import load_dataset
wmt16_test = load_dataset("ethansimrm/wmt16_biomed_test", use_auth_token=True)
wmt16_gold = load_dataset("ethansimrm/wmt16_biomed_gold", use_auth_token=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/ethansimrm--wmt16_biomed_test-195b3b7168424280/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/ethansimrm--wmt16_biomed_gold-4a81ce75a76006f6/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


/opt/conda/lib/python3.10/site-packages/datasets/packaged_modules/csv/csv.py:154: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  csv_file_reader = pd.read_csv(file, iterator=True, dtype=dtype, **self.config.read_csv_kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
source_sentences = wmt16_test["test"]["passage/0/sentence/0/text"]
target_sentences = wmt16_gold["train"]["passage/0/sentence/0/text"]

In [7]:
import evaluate
metric = evaluate.load("sacrebleu")

In [8]:
prefixed_ss = ["translate English to French: " + s for s in source_sentences]

In [9]:
from tqdm import tqdm
tqdm._instances.clear() #Stops newline printing error
predictions = [translator(s) for s in tqdm(prefixed_ss)]

100%|██████████| 500/500 [06:08<00:00,  1.36it/s]


In [10]:
preds = []
for p in predictions:
    preds.append(p[0]['translation_text'])

In [11]:
result = metric.compute(predictions=preds, references=target_sentences) #BLEU score for provided input and references
result = {"bleu": result["score"]}

In [12]:
result

{'bleu': 31.241005725769455}

In [13]:
translator_t5_small_base = pipeline("translation_en_to_fr", model = "t5-small") #Still 242MB in size
tqdm._instances.clear() #Stops newline printing error
predictions_t5_small_base = [translator_t5_small_base(s) for s in tqdm(prefixed_ss)]

  0%|          | 0/500 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
100%|██████████| 500/500 [07:56<00:00,  1.05it/s]


In [14]:
preds_t5_small_base  = []
for p in predictions_t5_small_base :
    preds_t5_small_base .append(p[0]['translation_text'])

In [15]:
result_t5_small_base = metric.compute(predictions=preds_t5_small_base, references=target_sentences) 
result_t5_small_base = {"bleu": result_t5_small_base["score"]}
result_t5_small_base #A clear improvement on the same test set.

{'bleu': 20.93195841231699}

In [16]:
translator_marian = pipeline("translation_en_to_fr", model = "Helsinki-NLP/opus-mt-en-fr") #301MB in size
tqdm._instances.clear() #Stops newline printing error
predictions_marian = [translator_marian(s) for s in tqdm(source_sentences)]


/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
100%|██████████| 500/500 [07:35<00:00,  1.10it/s]


In [17]:
preds_marian  = []
for p in predictions_marian :
    preds_marian.append(p[0]['translation_text'])

In [18]:
result_marian = metric.compute(predictions=preds_marian, references=target_sentences) 
result_marian = {"bleu": result_marian["score"]}
result_marian

{'bleu': 41.20155274065125}

In [19]:
from transformers import AutoModelForSeq2SeqLM

model_t5_trained = AutoModelForSeq2SeqLM.from_pretrained("ethansimrm/t5_small_prelim_emea_enfr")
print("My fine-tuned t5-small model has: " + str(model_t5_trained.num_parameters()) + " parameters")
model_t5_small = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
print("The pre-trained t5-small model has: " + str(model_t5_small.num_parameters()) + " parameters")
model_marian = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
print("The pre-trained Opus-MT model has: " + str(model_marian.num_parameters()) + " parameters")

My fine-tuned t5-small model has: 60506624 parameters
The pre-trained t5-small model has: 60506624 parameters
The pre-trained Opus-MT model has: 75133952 parameters
